In [1]:
import pandas as pd
import numpy as np

import handygenome
import handygenome.cnv.rdnacopy as rdnacopy

In [4]:
df = pd.DataFrame(
    {
        'Chromosome': np.repeat('chr1', 100),
        'Start': np.arange(100) + 1,
        'End': np.arange(100) + 2,
        'value': np.concatenate(
            [np.random.normal(2.5, 0.1, 40), np.random.normal(3.2, 0.1, 60)]
        ),
    }
)

In [5]:
seg = rdnacopy.run_segmentation(df)

Analyzing: Sample.1 


In [6]:
seg

,Chromosome,Start,End,num_data,mean_value
1,chr1,1,41,40.0,2.5298
2,chr1,41,101,60.0,3.2197


In [1]:
import pandas as pd
import numpy as np

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [8]:
arg_chrom = ro.StrVector(df['chrom'])
# arg_pos = ro.FloatVector(df['pos'])
arg_pos = ro.IntVector(df['pos'])
arg_value = ro.FloatVector(df['values'])

In [9]:
dnacopy = importr('DNAcopy')
cnaobj = dnacopy.CNA(arg_value, arg_chrom, arg_pos, data_type='logratio')
segresult = dnacopy.segment(cnaobj, **{'verbose': 1, 'undo.splits': 'sdundo'})

Analyzing: Sample.1 


In [10]:
segresult

data,[19]
output,[19]
segRows,[19]
call,[6]


In [11]:
with (ro.default_converter + pandas2ri.converter).context():
    seg_df = ro.conversion.get_conversion().rpy2py(segresult.rx2['output'])

In [12]:
seg_df

,ID,chrom,loc.start,loc.end,num.mark,seg.mean
1,Sample.1,chr1,1,40,40.0,2.4881
2,Sample.1,chr1,41,100,60.0,3.2012


In [1]:
import pysam

In [2]:
bam_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/14/LU-14.normal.bam'

In [3]:
bam = pysam.AlignmentFile(bam_path)

In [4]:
for idx, read in enumerate(bam.fetch()):
    if idx == 10:
        break
    print(read)
    

ST-E00130:582:HC7KJALXX:6:2108:26524:54612	81	#0	9998	0	95S56M	#1	182140535	0	CCTCACCCTAACCCTACCCCTAACCCTAACCCTAACCAGCACACGACCGCATACGATATCGAGTAACGTGACCCGAGTCCTAACGTGTGCTCTTACGATAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACC	array('B', [27, 15, 15, 6, 22, 27, 27, 15, 15, 15, 33, 22, 15, 15, 15, 15, 6, 15, 15, 15, 15, 22, 15, 22, 15, 6, 15, 27, 22, 6, 15, 6, 22, 15, 15, 15, 6, 15, 6, 6, 27, 6, 22, 6, 6, 22, 15, 6, 6, 22, 15, 22, 33, 15, 6, 15, 22, 27, 22, 15, 6, 15, 15, 15, 22, 15, 6, 6, 27, 27, 27, 15, 6, 6, 6, 33, 22, 15, 6, 15, 15, 15, 27, 6, 6, 15, 6, 22, 27, 27, 15, 27, 27, 15, 15, 22, 22, 15, 33, 22, 15, 27, 27, 27, 33, 27, 27, 27, 27, 27, 33, 27, 27, 27, 27, 27, 33, 27, 27, 27, 27, 27, 33, 27, 27, 27, 27, 27, 33, 27, 27, 27, 27, 27, 33, 27, 27, 27, 27, 27, 33, 27, 27, 27, 27, 33, 33, 33, 27, 27, 33])	[('SA', '12,95440,+,114S37M,0,1;'), ('MC', '52M99S'), ('MD', '56'), ('NM', 0), ('MQ', 0), ('AS', 56), ('XS', 53), ('RG', 'JKL-LU-14-blood-wgs-ILLUMINA')]
ST-E00130:582:HC7KJALXX

In [5]:
import pyranges as pr

In [9]:
f1 = pr.from_dict({'Chromosome': ['chr1', 'chr1', 'chr1'], 'Start': [3, 8, 5],
                   'End': [6, 9, 7], 'Name': ['interval1', 'interval3', 'interval2']})

f2 = pr.from_dict({'Chromosome': ['chr1', 'chr1'], 'Start': [1, 6],
                   'End': [2, 7], 'Name': ['a', 'b']})

In [11]:
f1

,Chromosome,Start,End,Name
0,chr1,3,6,interval1
1,chr1,8,9,interval3
2,chr1,5,7,interval2


In [12]:
f2

,Chromosome,Start,End,Name
0,chr1,1,2,a
1,chr1,6,7,b


In [10]:
f1.join(f2)

,Chromosome,Start,End,Name,Start_b,End_b,Name_b
0,chr1,5,7,interval2,6,7,b


In [14]:
f1.join(f2, how='left')

,Chromosome,Start,End,Name,Start_b,End_b,Name_b
0,chr1,5,7,interval2,6,7,b
1,chr1,3,6,interval1,-1,-1,-1
2,chr1,8,9,interval3,-1,-1,-1
